In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

# Exploration

In this first notebook we are going to explore the area of interest (AoI) we have selected for the workshop. Having our AoI, and within a range of dates we have defined, we are going to look for available Sentinel-2 imagery on the AoI through the [Sentinel Hub](https://www.sentinel-hub.com/) platform.

## Explore the AoI

First of all, let's explore the area of interest of this workshop. In this case we have chosen the [Boadella reservoir](https://es.wikipedia.org/wiki/Embalse_de_Darnius_Boadella) in Catalonia, Spain, which geometry is in the data folder as `data/boadella.geojson`. 

To do the exploration we are going to use [leafmap](https://leafmap.org/). As we want to make the first zoom to our AoI we need the centroid of the geoJSON file, which will also help us later to extract the bounding box of the images to download.

In [2]:
import leafmap
import geopandas as gpd

in_geojson = 'data/boadella.geojson'
gdf = gpd.read_file(in_geojson)

centroid_coords = gdf['geometry'].centroid
centroid = [centroid_coords.y.values[0], centroid_coords.x.values[0]]   # We are going to use the centroid later

m = leafmap.Map(center=centroid, zoom=13)
m.add_geojson(in_geojson, layer_name="Boadella reservoir")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Create bounding box

With the aim that all the images in the dataset have the same dimensions (512x512 pixels) we are going to use the centroid that we extracted before from the geoJSON and we are going to generate a bounding box of 512x512 pixels at 10m per pixel.

In [3]:
from eotdl.tools.tools import bbox_from_centroid

boadella_bbox = bbox_from_centroid(x=centroid[0], y=centroid[1], pixel_size=10, width=512, height=512)

Let's visualize the bounding box on a map!

In [4]:
from eotdl.tools.tools import bbox_to_polygon

# Create a polygon from the bbox
boadella_polygon = bbox_to_polygon(boadella_bbox)
# Create a GeoDataFrame from the polygon
gdf = gpd.GeoDataFrame(geometry=[boadella_polygon])
# Save the bounding box as a geoJSON file, if needed
# gdf.to_file('data/boadella_bbox.geojson', driver='GeoJSON')   # Uncomment to save the bbox as a GeoJSON file

m.add_geojson('data/boadella_bbox.geojson', layer_name="Boadella bbox")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Explore Sentinel-2 available imagery

### Sentinel Hub authentication

Now that he have our desired bounding box we can look for available Sentinel-2 imagery on it. As we are going to do it through the [Sentinel Hub](https://www.sentinel-hub.com/) platform, we need first to authenticate with our credentials. 

When a new user signs in, the EOTDL environment automatically receives credentials for Sentinel Hub, so you do not have to worry about them as they are managed by the EOTDL tools. Despite this, if you already had a Sentinel Hub account before accepting the EOTDL Terms and Conditions your Sentinel Hub credentials will NOT appear there, so you must retrieve them from you [Sentinel HUB dashboard](https://www.sentinel-hub.com/develop/dashboard/).

Knowing this, we can log in with our SH client in two ways.

If we did not already have an SH account, we can let the EOTDL take care of everything

In [1]:
from eotdl.access import SHClient

client = SHClient()

If we already had an account, we can save the SH credentials in our `.env` file or insert them here directly. If you prefer, you can algo set your SH credentials from your EOTDL [profile](https://www.eotdl.com/profile).

In [ ]:
from os import getenv
from eotdl.access import SHClient

sh_client_id = getenv('SH_CLIENT_ID')
sh_client_secret = getenv('SH_CLIENT_SECRET')

client = SHClient(sh_client_id=sh_client_id, 
                  sh_client_secret=sh_client_secret)

### Look for Sentinel-2 imagery

If we want to look for available Sentinel-2 imagery in our AoI, we must define a range of dates in which to search for the images. We have already defined a time interval for this workshop, which is in the `data/dates.csv` file.

In [6]:
import csv

dates = list()
with open("data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

dates[:5]

['2019-06-02', '2019-06-07', '2019-06-17', '2019-06-27', '2019-07-02']

Now we have a bounding box and a range of dates in which to search for available imagery, we have to transform them in a digestible format for the EOTDL environment. 

In this way, the platform simply needs a Python dictionary with format `'id': {'bounding_box': <bounding_box>, 'time_inverval': <time_interval>}` to be able not only to search for available data but also to download it. Being said that, let's create the dict.

> Although we have the specific dates, we are going to search for the entire time interval, just a demonstrator

In [9]:
boadella_search_dict = {
    'Boadella': {
        'bounding_box': boadella_bbox,
        'time_interval': (dates[0], dates[-1])
    }
}

boadella_available_data, boadella_not_available_data = client.get_available_data_by_location(boadella_search_dict,
                                                         sentinel_mission='sentinel-2')

The result is two Python dicts with the same format as the parameters dict: one with a list of dates with available imagery for the requested constellation and one dict with a list of dates without available imagery.

Let's check for some dates

In [10]:
dates = boadella_available_data['Boadella']['time_interval']
dates[:5]

[('2023-09-19', '2023-09-19'),
 ('2023-09-14', '2023-09-14'),
 ('2023-09-09', '2023-09-09'),
 ('2023-09-04', '2023-09-04'),
 ('2023-08-30', '2023-08-30')]

They make sense, as the [revisit time](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/#basic-facts) for Sentinel-2 is 5 days.

As a final step, let's check the number of dates with available images.

In [11]:
print(len(dates))

294


> Eventhough we can use the returned dict with dates for the download section, we already have a list of dates with available data we know is usable for the workshop, like imagery with no clouds, good conditions, and so on.

To sum up this section, we have explored our AoI, generated a bounding box and a time interval in which to look for imagery and searched for Sentinel-2 imagery.

Let's continue in the [01_download](./01_download.ipynb) notebook and download the images!